In [1]:
import os
import numpy as np
from skimage import io
import keras.backend as K
from unet import unet, preproc

Using TensorFlow backend.


In [2]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# Model prediction and ensembling

## Set parameters 
__Laboratory, consensus strategy, and model weights__
- `LAB`: one of `inns1`, `inns2`, `mue`, `wue1`, `wue2`
- `INIT`: original model initialization, e.g., `from-scratch` available for _lab_wue1_, `fine-tuned` for other labs
- `ENSEMBLE`: Ensemble name (`consensus1` available)
- `CHECKPOINT_DIR`: relative path to stored model weights

In [7]:
LAB = 'wue1'
INIT = 'from-scratch'
ENSEMBLE_LIST = ['consensus1', 'consensus2', 'consensus3']
CHECKPOINT_DIR = 'model_library'
MASK = 'cFOS'
IMAGE = '_red'

# Other parameters as used in our paper
BATCH_SIZE = 4
TILE_SHAPE = (540,540)
PADDING = (184,184)

## Load and prepare data

(data not included in github repository)

In [8]:
DATA_PATH = "_KO_DATA/img_cfos/" 
# Get image IDs
FILE_IDS = [x.rsplit('_',1)[0] for x in os.listdir(DATA_PATH)]

## Loop

In [9]:
for file_ids in chunks(FILE_IDS, 150):
    # Load images
    images = [np.expand_dims(io.imread(os.path.join(DATA_PATH, x), as_gray=True), axis=2)
              for x in [s + IMAGE +'.tif' for s in file_ids]]

    # Create generator
    data = [{'rawdata': img, 'element_size_um': [1,1]} for img in images]
    tile_generator = preproc.TileGenerator(data, TILE_SHAPE, PADDING)


    ## Compute masks and ensemble predictions
    for ENSEMBLE in ENSEMBLE_LIST:
        
        CHECKPOINT_PATH = '{}/lab-{}/{}/{}'.format(CHECKPOINT_DIR, LAB, INIT, ENSEMBLE)
        MODELS = [x for x in os.listdir(CHECKPOINT_PATH) if x.startswith('model')]
        # Path to save predictions
        
        path = os.path.join('pred_masks', 'lab-'+LAB, ENSEMBLE)
        softmax_dict = {}
        for model in MODELS:

            # Get checkpoint names
            cp_name = os.listdir(os.path.join(CHECKPOINT_PATH, model))[0]
            mod_cp = os.path.join(model, cp_name)

            # Create unet model
            cp_model = unet.Unet2D(snapshot= os.path.join(CHECKPOINT_PATH, mod_cp))

            # Predict new masks from selected checkpoints
            predictions = cp_model.predict(tile_generator)
            softmax_dict[mod_cp] = [predictions[0][i][:,:,1] for i in range(len(file_ids))]

            # Free GPU RAM
            sess = K.get_session()
            K.clear_session()
            sess.close()  

        # Calculate and save softmax average
        for i, idx in enumerate(file_ids):
            stack_tmp = [softmax_dict[cp][i] for cp in softmax_dict]
            stack_tmp = np.mean(stack_tmp, axis=0)
            file_name = idx + '_' +  MASK + '.png'
            bin_path = os.path.join(path, 'ensemble', LAB+'_'+ENSEMBLE+'_ensemble')
            if not os.path.isdir(bin_path): os.makedirs(bin_path)
            io.imsave(os.path.join(bin_path, file_name), (stack_tmp > 0.5).astype('uint8') * 255)

  1%|          | 1/150 [00:00<00:16,  8.90it/s]

Processing test samples


100%|██████████| 150/150 [00:14<00:00, 10.43it/s]

100%|██████████| 1350/1350 [01:10<00:00, 18.92it/s]
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/consensus1/ensemble/wue1_consensus1_ensemble/0789_cFOS.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/consensus1/ensemble/wue1_consensus1_ensemble/1596_cFOS.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/consensus1/ensemble/wue1_consensus1_ensemble/1713_cFOS.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/consensus1/ensemble/wue1_consensus1_ensemble/1591_cFOS.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: p

Processing test samples


...

100%|██████████| 1350/1350 [01:09<00:00, 19.14it/s]
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/consensus1/ensemble/wue1_consensus1_ensemble/0818_cFOS.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/consensus1/ensemble/wue1_consensus1_ensemble/0153_cFOS.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/consensus1/ensemble/wue1_consensus1_ensemble/2081_cFOS.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/consensus1/ensemble/wue1_consensus1_ensemble/1862_cFOS.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: p

Processing test samples


100%|██████████| 1350/1350 [01:09<00:00, 19.51it/s]
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/consensus1/ensemble/wue1_consensus1_ensemble/1891_cFOS.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/consensus1/ensemble/wue1_consensus1_ensemble/1217_cFOS.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/consensus1/ensemble/wue1_consensus1_ensemble/0713_cFOS.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/consensus1/ensemble/wue1_consensus1_ensemble/1914_cFOS.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: p

Processing test samples


100%|██████████| 63/63 [00:03<00:00, 19.36it/s]
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/consensus1/ensemble/wue1_consensus1_ensemble/1382_cFOS.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/consensus1/ensemble/wue1_consensus1_ensemble/1383_cFOS.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/consensus1/ensemble/wue1_consensus1_ensemble/0841_cFOS.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/consensus1/ensemble/wue1_consensus1_ensemble/1343_cFOS.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_

In [10]:
LAB = 'wue1'
INIT = 'from-scratch'
ENSEMBLE_LIST = ['parv-consensus1', 'parv-consensus2', 'parv-consensus3']
CHECKPOINT_DIR = 'model_library'
MASK = 'Parv'
IMAGE = '_green'
DATA_PATH = "_KO_DATA/img_parv/" 
# Get image IDs
FILE_IDS = [x.rsplit('_',1)[0] for x in os.listdir(DATA_PATH)]

# Other parameters as used in our paper
BATCH_SIZE = 4
TILE_SHAPE = (540,540)
PADDING = (184,184)

## Loop

In [11]:
for file_ids in chunks(FILE_IDS, 150):
    # Load images
    images = [np.expand_dims(io.imread(os.path.join(DATA_PATH, x), as_gray=True), axis=2)
              for x in [s + IMAGE +'.tif' for s in file_ids]]

    # Create generator
    data = [{'rawdata': img, 'element_size_um': [1,1]} for img in images]
    tile_generator = preproc.TileGenerator(data, TILE_SHAPE, PADDING)


    ## Compute masks and ensemble predictions
    for ENSEMBLE in ENSEMBLE_LIST:
        
        CHECKPOINT_PATH = '{}/lab-{}/{}/{}'.format(CHECKPOINT_DIR, LAB, INIT, ENSEMBLE)
        MODELS = [x for x in os.listdir(CHECKPOINT_PATH) if x.startswith('model')]
        # Path to save predictions
        
        path = os.path.join('pred_masks', 'lab-'+LAB, ENSEMBLE)
        softmax_dict = {}
        for model in MODELS:

            # Get checkpoint names
            cp_name = os.listdir(os.path.join(CHECKPOINT_PATH, model))[0]
            mod_cp = os.path.join(model, cp_name)

            # Create unet model
            cp_model = unet.Unet2D(snapshot= os.path.join(CHECKPOINT_PATH, mod_cp))

            # Predict new masks from selected checkpoints
            predictions = cp_model.predict(tile_generator)
            softmax_dict[mod_cp] = [predictions[0][i][:,:,1] for i in range(len(file_ids))]

            # Free GPU RAM
            sess = K.get_session()
            K.clear_session()
            sess.close()  

        # Calculate and save softmax average
        for i, idx in enumerate(file_ids):
            stack_tmp = [softmax_dict[cp][i] for cp in softmax_dict]
            stack_tmp = np.mean(stack_tmp, axis=0)
            file_name = idx + '_' +  MASK + '.png'
            bin_path = os.path.join(path, 'ensemble', LAB+'_'+ENSEMBLE+'_ensemble')
            if not os.path.isdir(bin_path): os.makedirs(bin_path)
            io.imsave(os.path.join(bin_path, file_name), (stack_tmp > 0.5).astype('uint8') * 255)

  1%|▏         | 2/150 [00:00<00:14, 10.44it/s]

Processing test samples


100%|██████████| 1350/1350 [01:09<00:00, 19.48it/s]
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/0281_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/2038_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/2268_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/1350_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/

Processing test samples


100%|██████████| 1350/1350 [01:09<00:00, 19.46it/s]
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/2107_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/1850_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/1217_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/0103_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/

...

100%|██████████| 1350/1350 [01:09<00:00, 19.14it/s]
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/1395_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/1491_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/0777_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/2007_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/

Processing test samples


100%|██████████| 1350/1350 [01:09<00:00, 19.23it/s]
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/1851_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/1497_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/2133_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/1859_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/

Processing test samples


100%|██████████| 63/63 [00:03<00:00, 19.42it/s]
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/2213_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/1313_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/1599_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipykernel_launcher.py:45: UserWarning: pred_masks/lab-wue1/parv-consensus1/ensemble/wue1_parv-consensus1_ensemble/1930_Parv.png is a low contrast image
/home/firstusr/anaconda3/envs/keras-gpu/lib/python3.7/site-packages/ipyk